First, we need to import the ImageFont, ImageDraw, and Image modules from PIL

In [1]:
from PIL import ImageFont, ImageDraw, Image

Now, we have to create a blank image object. For this, we first need to create a three-dimensional (for the three color channels) numpy zeros array

In [2]:
import numpy as np
img = np.zeros(shape=(25, 60, 3), dtype=np.uint8)

This gives us an array where each element represents a pixel of the image, the size of the image being 60x25 pixels:

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

To create an image from this array, we call Image’s fromarray() method

In [3]:
img_pil = Image.fromarray(img+123)
img_pil

The image is black because each pixel has a value of (0, 0, 0), which corresponds to the brightness of the red, green, and blue pixels. Brightness has a value between 0 (darkest) and 255 (brightest). To get a white image, just add 255 to the image array

In [4]:
img_pil = Image.fromarray(img+255)
img_pil

Now, we need to draw text over our canvas. For this, we first create a drawing interface using ImageDraw's Draw() function on our canvas

In [5]:
draw = ImageDraw.Draw(img_pil)

Then, we can use Draw’s text() method to write text over the canvas. The draw method takes the following arguments:
1. xy: Starting coordinates for the text as a tuple (x, y).
2. text: The text to be drawn.
3. font: The font to be used. This is a FreeType or OpenType font used by PIL, and this is where ImageFont comes into play.
4. fill: The fill color for the text, expressed as (R, G, B).

We need to have our font object ready for this. 
We will use ImageDraw's truetype() function

In [6]:
font = ImageFont.truetype(font = 'arial', size=12)

We have used the font name, but truetype can also take the font path on the system. We’ll get to this later.

Now that we have the font, let us add the text and see what the CAPTCHA looks like

In [7]:
draw.text((5,10), 'Capture', font = font, fill = (0,0,0))
img_pil

We can also add lines to the image to confuse any robot trying to break into a human-only system!

In [8]:
draw.line([(0,0),(60,25),(0,25),(60,0)], width = 1, fill = (0,0,0))
img_pil

The tuples indicate the starting and ending pixels of the lines. The first line starts at (0, 0) and ends at (60, 25).
A better way to see this is to use OpenCV’s imshow() method, which renders an image from an array

In [9]:
import cv2
cv2.imshow('Capture',np.array(img_pil))
cv2.waitKey() #Displays the image till a key is pressed
cv2.destroyAllWindows()

Adding Noise

Our basic CAPTCHA is now ready, but it looks very clean and readable compared to CAPTCHAs you see online. Let’s add some noise!

The easiest way to do this is by randomly adding white and black pixels to the image. This is called salt and pepper noise.

First, we have to create an array from our image

In [10]:
img = np.array(img_pil)

Then, we define a threshold of pixels to be altered. We will keep it at 0.05 (5%). The code to add noise is a simple nested for-loop with a randomly generated number (between 0-1) to determine if noise will be added to the specific pixel

In [11]:
import random
thresh = 0.05

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        rdn = random.random()
        if rdn < thresh:
            img[i][j] = 0
        elif rdn > 1-thresh:
            img[i][j] = 255

Here, if the random number is lower than the threshold (0.05), we make it white. If it’s greater than 0.95, we make it black. Otherwise, we let it remain as is. The output after adding noise looks like this:

The pepper noise (black pixels) is pretty evident, but a few pixels on the text that were black earlier are now white (notice the breaks in the lines).

We can add more noise by blurring the image, which will make the noise a bit more spread out. With cv2, this is just a one-liner!

In [12]:
img_blurred = cv2.blur(img,(2,2))

Here, (2,2) is the size of the kernel used to smoothen the image. Read more about it in OpenCV’s documentation

In [13]:
size = random.randint(10,16)
length = random.randint(4,8)

Randomizing Everything!

Our basic code is ready… but the “Random” in the “Random Text CAPTCHAs” is missing. Let us randomize most of the things so that each CAPTCHA is truly unique!

So, what can be randomized? Well, everything, but we will concentrate on the image size, the text (string, font, font size, color), line position, line color, noise threshold, noise intensity, and blur intensity.

The image size will depend on the length of the text and the font size. So we first make a font-size variable and the length of the string:

In [14]:
img = np.zeros(((size*2)+5, length*size, 3), np.uint8)

Now, to randomize the fonts, we can pick some from the system fonts path using the glob library. I am using only variations of the Arial font.

In [15]:
import glob
font_path = r'C:\Windows\Fonts'
fonts = glob.glob(font_path+'\\ari*.ttf')
fonts

['C:\\Windows\\Fonts\\arial.ttf',
 'C:\\Windows\\Fonts\\arialbd.ttf',
 'C:\\Windows\\Fonts\\arialbi.ttf',
 'C:\\Windows\\Fonts\\ariali.ttf',
 'C:\\Windows\\Fonts\\ARIALN.TTF',
 'C:\\Windows\\Fonts\\ARIALNB.TTF',
 'C:\\Windows\\Fonts\\ARIALNBI.TTF',
 'C:\\Windows\\Fonts\\ARIALNI.TTF',
 'C:\\Windows\\Fonts\\ARIALUNI.TTF',
 'C:\\Windows\\Fonts\\ariblk.ttf']

Then we can choose a random font by using random.choice()

In [16]:
font = random.choice(fonts)

Now to the text. We generate a random sequence of ASCII alphanumeric characters given the length of the string

In [17]:
import string
text = ''.join(random.choice(string.ascii_uppercase + string.digits + string.ascii_lowercase) for _ in range(length))

Since the length is already random, we just call this function by passing length

In [18]:
from django.utils.crypto import get_random_string
print(length)
text = get_random_string(length)
print(text)

6
WUuWGT


For the noise threshold, we can set it to any random value between 1% and 5%

In [21]:
thresh = random.randint(1,5)/100

For noise intensity, instead of using absolute white and black for salt and pepper pixels, respectively, we can set them to random bright and dark shades

In [22]:
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        rdn = random.random()
        if rdn < thresh:
            img[i][j] = random.randint(0,123) #dark pixels
        elif rdn > 1-thresh:
            img[i][j] = random.randint(123,255) #bright pixels

Finally, to control the blur intensity, we can randomize the kernel size from (2, 2) to 
(int(size/random.randint(5,10)),int(size/random.randint(5,10))). A range between 5 and 10 is chosen as any value less than 5 makes the image too blurred to be legible, while a value greater than 10 doesn’t have much of a blurring effect.

In [23]:
img = cv2.blur(img,(int(size/7),int(size/7)))

Putting It All Together

In [24]:
# Setting up the canvas
size = random.randint(10,16)
length = random.randint(4,8)
img = np.zeros(((size*2)+5, length*size, 3), np.uint8)
img_pil = Image.fromarray(img+255)

# Drawing text and lines
font = ImageFont.truetype(random.choice(fonts), size)
draw = ImageDraw.Draw(img_pil)
text = ''.join(
    random.choice(string.ascii_uppercase + string.digits + string.ascii_lowercase) 
               for _ in range(length))
draw.text((5, 10), text, font=font, 
          fill=(random.randint(0,255), random.randint(0,255), random.randint(0,255)))
draw.line([(random.choice(range(length*size)), random.choice(range((size*2)+5)))
           ,(random.choice(range(length*size)), random.choice(range((size*2)+5)))]
          , width=1, fill=(random.randint(0,255), random.randint(0,255), random.randint(0,255)))

# Adding noise and blur
img = np.array(img_pil)
thresh = random.randint(1,5)/100
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        rdn = random.random()
        if rdn < thresh:
            img[i][j] = random.randint(0,123)
        elif rdn > 1-thresh:
            img[i][j] = random.randint(123,255)
img = cv2.blur(img,(int(size/random.randint(5,10)),int(size/random.randint(5,10))))

#Displaying image
cv2.imshow(f"{text}", img)
cv2.waitKey()
cv2.destroyAllWindows()
#cv2.imwrite(f"{OUTPUT_FOLDER}/{text}.png", img) #if you want to save the image

Applications : -

This can be used to generate CAPTCHAs. You can save the images with the text as the filename to be matched. Another novel application can be to generate a large number of labeled images to train your OCR models. Put the code above in a loop, and you’ll get as many images as you want!

In [ ]:
# https://medium.com/better-programming/how-to-generate-random-text-captchas-using-python-e734dd2d7a51